## Re-compile LGBM with GPU support

In [1]:
!rm -r /opt/conda/lib/python3.6/site-packages/lightgbm
!git clone --recursive https://github.com/Microsoft/LightGBM

rm: cannot remove '/opt/conda/lib/python3.6/site-packages/lightgbm': No such file or directory
Cloning into 'LightGBM'...
remote: Enumerating objects: 23671, done.
remote: Counting objects: 100% (1347/1347), done.
remote: Compressing objects: 100% (731/731), done.
remote: Total 23671 (delta 851), reused 979 (delta 584), pack-reused 22324
Receiving objects: 100% (23671/23671), 18.14 MiB | 22.41 MiB/s, done.
Resolving deltas: 100% (17244/17244), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/kaggle/working/LightGBM/external_libs/compute'...
remote: 

In [2]:
!apt-get install -y -qq libboost-all-dev

In [3]:
%%bash
cd LightGBM
rm -r build
mkdir build
cd build
cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
make -j$(nproc)

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Looking for CL_VERSION_2_2
-- Looking for CL_VERSION_2_2 - not found
-- Looking for CL_VERSION_2_1
-- Looking for CL_VERSION_2_1 - not found
-- Looking for CL_VERSION_2_0
-- Looking for CL_VERSION_2_0 - not found
-- Looking for CL_VERSION_1_2
-- Looking

rm: cannot remove 'build': No such file or directory
cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/u

In [4]:
!cd LightGBM/python-package/;python3 setup.py install --precompile

running install
running build
running build_py
creating build
creating build/lib
creating build/lib/lightgbm
copying lightgbm/engine.py -> build/lib/lightgbm
copying lightgbm/basic.py -> build/lib/lightgbm
copying lightgbm/__init__.py -> build/lib/lightgbm
copying lightgbm/compat.py -> build/lib/lightgbm
copying lightgbm/libpath.py -> build/lib/lightgbm
copying lightgbm/plotting.py -> build/lib/lightgbm
copying lightgbm/callback.py -> build/lib/lightgbm
copying lightgbm/sklearn.py -> build/lib/lightgbm
copying lightgbm/dask.py -> build/lib/lightgbm
running egg_info
creating lightgbm.egg-info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-level names to lightgbm.egg-info/top_level.txt
writing manifest file 'lightgbm.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
no previously-included directories found matching 'build'
writing manifest file 'lightgbm.e

In [5]:
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
!rm -r LightGBM

## Loading dataset

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import numpy as np
import time
import lightgbm as lgb

from sklearn.model_selection import GroupKFold 
from sklearn.model_selection import  KFold
from sklearn import metrics

In [7]:
train = pd.read_csv("../input/ventilator-pressure-prediction/train.csv")
test = pd.read_csv("../input/ventilator-pressure-prediction/test.csv")
submission = pd.read_csv("../input/ventilator-pressure-prediction/sample_submission.csv")
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987


In [8]:
train_preds = pd.DataFrame(train, columns=['id'])
train_preds['pressure_preds'] = ""
train_preds.head()

,id,pressure_preds
0,1,
1,2,
2,3,
3,4,
4,5,


In [9]:
def add_features(df):
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    df['cross']= df['u_in']*df['u_out']
    df['cross2']= df['time_step']*df['u_out']
    
    df['u_in_cumsum'] = df.groupby(['breath_id'])['u_in'].cumsum()
    df['time_step_cumsum'] = df.groupby(['breath_id'])['time_step'].cumsum()
    
    df['u_in_1st_derivative'] = (df['u_in'].diff().fillna(0) / df['time_step'].diff().fillna(0)).fillna(0)
    df['expand_mean_1sr_der'] = df.groupby('breath_id')['u_in_1st_derivative'].expanding(2).mean().reset_index(level=0,drop=True)
    df['u_in_1st_der_mean10'] = df.groupby('breath_id')['u_in_1st_derivative'].rolling(window=10, min_periods=1).mean().reset_index(level=0,drop=True)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4).fillna(0).reset_index(level=0,drop=True)
    df['u_in_lag-4'] = df.groupby('breath_id')['u_in'].shift(-4).fillna(0).reset_index(level=0,drop=True)
    
    df['time_diff'] = df['time_step']-df['time_step'].shift(1)
    df['power'] = df['time_diff']*df['u_in']
    df['power_cumsum'] = df.groupby(['breath_id'])['power'].cumsum()
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_in__min'] = df.groupby(['breath_id'])['u_in'].transform('min')
                                                     
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['RC'] = df['R']+df['C']
    df = pd.get_dummies(df)
    
    df['last_value_u_in'] = df.groupby('breath_id')['u_in'].transform('last')
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']

    df['ewm_u_in_mean'] = df.groupby('breath_id')['u_in'].ewm(halflife=10).mean().reset_index(level=0,drop=True)
    df['ewm_u_in_std'] = df.groupby('breath_id')['u_in'].ewm(halflife=10).std().reset_index(level=0,drop=True)
    df['ewm_u_in_corr'] = df.groupby('breath_id')['u_in'].ewm(halflife=10).corr().reset_index(level=0,drop=True)
    
    df['delta_u_in'] = abs(df.groupby(df['breath_id'])['u_in'].diff().fillna(0)).reset_index(level=0,drop=True)
    df['delta_u_in_exp'] = df.groupby(df['breath_id'])['delta_u_in'].rolling(window=10, min_periods=1).mean().reset_index(level=0,drop=True)
    df['delta_rolling_10_mean'] = df.groupby('breath_id')['delta_u_in'].rolling(window=10, min_periods=1).mean().reset_index(level=0,drop=True)
    df['delta_rolling_10_max'] = df.groupby('breath_id')['delta_u_in'].rolling(window=10, min_periods=1).max().reset_index(level=0,drop=True)
   
    df['work']=((df['u_in'] + df['u_in'].shift(1).fillna(0))/2 * df['time_step'].diff().fillna(0)).clip(0,)
    df['work_roll_10']=df.groupby(df['breath_id'])['work'].rolling(window=10, min_periods=1).sum().reset_index(level=0,drop=True)
    df['work_roll_15']=df.groupby(df['breath_id'])['work'].rolling(window=15, min_periods=1).sum().reset_index(level=0,drop=True)
      
    df['u_in_rol_q0.1'] = df.groupby(df['breath_id'])['u_in'].rolling(window=10, min_periods=1).quantile(0.1).reset_index(level=0,drop=True)
    df['u_in_rol_q0.25'] = df.groupby(df['breath_id'])['u_in'].rolling(window=10, min_periods=1).quantile(0.25).reset_index(level=0,drop=True)
    df['u_in_rol_q0.5'] = df.groupby(df['breath_id'])['u_in'].rolling(window=10, min_periods=1).quantile(0.5).reset_index(level=0,drop=True)
    df['u_in_rol_q0.75'] = df.groupby(df['breath_id'])['u_in'].rolling(window=10, min_periods=1).quantile(0.75).reset_index(level=0,drop=True)
    df['u_in_rol_q0.9'] = df.groupby(df['breath_id'])['u_in'].rolling(window=10, min_periods=1).quantile(0.9).reset_index(level=0,drop=True)     
    
    df = df.fillna(0)
    return df

train = add_features(train)
test = add_features(test)

In [10]:
train = train[train['u_out'] < 1].reset_index(drop=True)

In [11]:
!nvidia-smi

Mon Oct 25 19:28:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
scores = []
feature_importance = pd.DataFrame()
models = []
columns = [col for col in train.columns if col not in ['id', 'breath_id', 'pressure','u_out']]
columns = ['time_step', 'u_in', 'area', 'u_in_cumsum', 'time_step_cumsum', 
           'u_in_1st_derivative', 'expand_mean_1sr_der', 'u_in_1st_der_mean10',
           'u_in_lag4', 'u_in_lag-4', 'time_diff', 'power_cumsum', 'u_in_lag2',
           'u_in_lag3', 'breath_id__u_in__max', 'C_50', 'RC_2020', 'RC_2050', 
           'RC_5010', 'RC_510', 'RC_520', 'RC_550', 'last_value_u_in', 'u_in_lag_back1', 
           'u_in_lag_back2', 'u_in_lag_back3', 'u_in_diff1', 'u_in_diff2', 
           'breath_id__u_in__diffmean', 'ewm_u_in_mean', 'ewm_u_in_std', 'ewm_u_in_corr', 
           'delta_u_in', 'delta_rolling_10_mean', 'delta_rolling_10_max', 'work',
           'work_roll_10', 'work_roll_15', 'u_in_rol_q0.1', 'u_in_rol_q0.25', 
           'u_in_rol_q0.5', 'u_in_rol_q0.75', 'u_in_rol_q0.9', 'R_5', 'RC_5050', 
           'breath_id__u_in__diffmax']
X = train[columns]
y = train['pressure']
params = {'objective': 'regression',
          'learning_rate': 0.2, #0.35,
          "boosting_type": "gbdt",
          'min_data_in_leaf':30,
          'max_bin': 119, #600,
#           'max_bin': 700, #600,
          'num_leaves':1200,
          "metric": 'mse', #'mae',
          'n_jobs': -1,
          'device': 'gpu',
          'gpu_platform_id': 0,
          'gpu_device_id': 0
         }
folds = GroupKFold(n_splits=5)
for fold_n, (train_index, valid_index) in enumerate(folds.split(train, y, groups=train['breath_id'])):
    print(f'Fold {fold_n} started at {time.ctime()}')
    X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    model = lgb.LGBMRegressor(**params, n_estimators=6000)
    model.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)],
            verbose=100, early_stopping_rounds=25)
    score = metrics.mean_absolute_error(y_valid, model.predict(X_valid))
    
    models.append(model)
    scores.append(score)

    #fold_importance = pd.DataFrame()
    #fold_importance["feature"] = columns
    #fold_importance["importance"] = model.feature_importances_
    #fold_importance["fold"] = fold_n + 1
    #feature_importance = pd.concat([feature_importance, fold_importance], axis=0)
    
print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))

Fold 0 started at Mon Oct 25 19:28:59 2021


/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[100]	training's l2: 0.448807	valid_1's l2: 0.890518
[200]	training's l2: 0.297141	valid_1's l2: 0.834181
[300]	training's l2: 0.220501	valid_1's l2: 0.810058
[400]	training's l2: 0.169082	valid_1's l2: 0.791713
[500]	training's l2: 0.135122	valid_1's l2: 0.782432
[600]	training's l2: 0.109881	valid_1's l2: 0.777276
[700]	training's l2: 0.0914083	valid_1's l2: 0.772122
[800]	training's l2: 0.0772555	valid_1's l2: 0.768338
[900]	training's l2: 0.0658602	valid_1's l2: 0.765304
[1000]	training's l2: 0.0565063	valid_1's l2: 0.762754
[1100]	training's l2: 0.0488769	valid_1's l2: 0.761159
[1200]	training's l2: 0.0428541	valid_1's l2: 0.759764
[1300]	training's l2: 0.0375125	valid_1's l2: 0.758666
[1400]	training's l2: 0.0330198	valid_1's l2: 0.757922
Fold 1 started at Mon Oct 25 19:35:52 2021


/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[100]	training's l2: 0.446879	valid_1's l2: 0.862382
[200]	training's l2: 0.298434	valid_1's l2: 0.805658
[300]	training's l2: 0.220002	valid_1's l2: 0.779603
[400]	training's l2: 0.171068	valid_1's l2: 0.766908
[500]	training's l2: 0.137278	valid_1's l2: 0.757681
[600]	training's l2: 0.112158	valid_1's l2: 0.751021
[700]	training's l2: 0.0930351	valid_1's l2: 0.745731
[800]	training's l2: 0.0783848	valid_1's l2: 0.741847
[900]	training's l2: 0.0669215	valid_1's l2: 0.738424
[1000]	training's l2: 0.0571318	valid_1's l2: 0.736116
[1100]	training's l2: 0.0495732	valid_1's l2: 0.734538
[1200]	training's l2: 0.0429613	valid_1's l2: 0.732733
[1300]	training's l2: 0.0375401	valid_1's l2: 0.731315
[1400]	training's l2: 0.0329784	valid_1's l2: 0.730434
[1500]	training's l2: 0.0290533	valid_1's l2: 0.729584
[1600]	training's l2: 0.0255991	valid_1's l2: 0.728644
[1700]	traini

...

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[100]	training's l2: 0.443007	valid_1's l2: 0.912536
[200]	training's l2: 0.295205	valid_1's l2: 0.857273
[300]	training's l2: 0.2151	valid_1's l2: 0.829078
[400]	training's l2: 0.168068	valid_1's l2: 0.815299
[500]	training's l2: 0.135086	valid_1's l2: 0.806179
[600]	training's l2: 0.109818	valid_1's l2: 0.799185
[700]	training's l2: 0.0907417	valid_1's l2: 0.794085
[800]	training's l2: 0.0773845	valid_1's l2: 0.790572
[900]	training's l2: 0.0661046	valid_1's l2: 0.787932
[1000]	training's l2: 0.0570929	valid_1's l2: 0.786192
[1100]	training's l2: 0.0490493	valid_1's l2: 0.78492
[1200]	training's l2: 0.0428317	valid_1's l2: 0.783572
[1300]	training's l2: 0.0376196	valid_1's l2: 0.782406
[1400]	training's l2: 0.0328584	valid_1's l2: 0.781442
[1500]	training's l2: 0.0287291	valid_1's l2: 0.780555
[1600]	training's l2: 0.0253175	valid_1's l2: 0.779658
[1700]	training'

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[100]	training's l2: 0.448141	valid_1's l2: 0.840021
[200]	training's l2: 0.294764	valid_1's l2: 0.779376
[300]	training's l2: 0.221541	valid_1's l2: 0.754467
[400]	training's l2: 0.170576	valid_1's l2: 0.74312
[500]	training's l2: 0.137058	valid_1's l2: 0.733197
[600]	training's l2: 0.111857	valid_1's l2: 0.726538
[700]	training's l2: 0.0936519	valid_1's l2: 0.722382
[800]	training's l2: 0.0782799	valid_1's l2: 0.718076
[900]	training's l2: 0.0663528	valid_1's l2: 0.715012
[1000]	training's l2: 0.0568272	valid_1's l2: 0.712369
[1100]	training's l2: 0.0491651	valid_1's l2: 0.710455
[1200]	training's l2: 0.0427595	valid_1's l2: 0.709106
[1300]	training's l2: 0.0375085	valid_1's l2: 0.707781
[1400]	training's l2: 0.032778	valid_1's l2: 0.706769
[1500]	training's l2: 0.0289327	valid_1's l2: 0.705723
[1600]	training's l2: 0.0254183	valid_1's l2: 0.705101
[1700]	training

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[100]	training's l2: 0.454276	valid_1's l2: 0.907656
[200]	training's l2: 0.299849	valid_1's l2: 0.847397
[300]	training's l2: 0.221371	valid_1's l2: 0.821542
[400]	training's l2: 0.172192	valid_1's l2: 0.806821
[500]	training's l2: 0.137519	valid_1's l2: 0.796346
[600]	training's l2: 0.112921	valid_1's l2: 0.790495
[700]	training's l2: 0.093165	valid_1's l2: 0.785847
[800]	training's l2: 0.0777626	valid_1's l2: 0.781957
[900]	training's l2: 0.0660792	valid_1's l2: 0.779212
[1000]	training's l2: 0.0566507	valid_1's l2: 0.776506
[1100]	training's l2: 0.0491616	valid_1's l2: 0.774733
[1200]	training's l2: 0.0425553	valid_1's l2: 0.773417
[1300]	training's l2: 0.0371039	valid_1's l2: 0.772255
[1400]	training's l2: 0.0328029	valid_1's l2: 0.771488
[1500]	training's l2: 0.0290449	valid_1's l2: 0.770474
[1600]	training's l2: 0.0256669	valid_1's l2: 0.769735
[1700]	trainin

In [13]:
for num, model in enumerate(models):
    submission['pressure'] += model.predict(test[columns])
#     model.save_model(f'./LGBM_model_{num+1}.txt')
    train_preds['pressure_preds'] += model.predict(train[columns])
submission['pressure'] /= 5
submission.to_csv('submission.csv', index=False)
train_preds['pressure_preds'] /= 5 
train_preds.to_csv('train_preds.csv',index=False)


# # ENSEMBLE FOLDS WITH MEDIAN
# #取中位数
# submission["pressure"] = np.median(np.vstack(test_preds),axis=0)
# submission.to_csv('submission_median.csv', index=False)


# # ENSEMBLE FOLDS WITH MEDIAN AND ROUND PREDICTIONS
# submission["pressure"] =\
#     np.round( (submission.pressure - pressure_min)/pressure_step ) * pressure_step + pressure_min
# submission.pressure = np.clip(submission.pressure, pressure_min, pressure_max)
# submission.to_csv('submission_median_round.csv', index=False)

ValueError: operands could not be broadcast together with shapes (6036000,) (2290968,) 